# 최종 정리

## 요구사항

이 것을 왜 하냐? 계속 구글 스프레스 시트로 반복 작업을 하고 싶지 않았기 때문이다.

- Ajax 로 특정 시군구 긁어오기
- 정렬, 복사, 붙여넣기 등
- 스프레트 시트에 지역-탭 별로 나누기

이 정보의 최종 활용자는? 50대 컴퓨터에 익숙치 않은 어른. 그가 이 프로그램을 조작하진 않는다. 그리고 난 그에게 엑셀 파일만 주면 된다.

몇 개의 필드를 숨기거나 하는 작업을 해도 된다. 그렇지만, 자료를 업데이트할 때마다 매번 자료의 형태를 가공해야하는 작업이 싫다.

즉, 최종적으로 나는

1. 내가 원하는 지역 경기도, 인천, 서울의 모든 아파트 정보를
    - 각 페이지에
    - 함수 한 번으로 가져오고
    - 저장한 다음에
    - 이것을 아래의 형태로 가공해서 각 부천,시흥시,광명시,구로구,양천구,부평구 페이지에 저장한다.
    - 양식을 붙여넣을 수 있기 때문에.

2. 최신 정보를 갱신할 때는.
    - 기존의 정보의 명칭과 새로 받아온 것의 명칭을 비교해서
    - 정보가 틀릴 때는 최신 것으로 업데이트하고
    - 정보가 없을 때는 새로 추가하며
    - 정보가 같을 때는 아무일도 하지 않는다.

그렇게 되면 나는 이미 가공된 자료의 필드 몇개를 숨기거나 양식을 붙여넣는 수준으로 작업을 편하게 할 수 있다.

연번|명칭|법정동|난방|관리사무실|팩스|준공일|방식|연차|동수|방광|요일|시간|금액|선입|문어|매수|1차|2차|비고
--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--|--
ㅇ|

## 피드백

비판과 재비판을 함 해보려고 한다.

**원하는 지역의 아파트 코드만 가져온다**

1. 필요한 건 오직 특정 시군구 정보 뿐이다.
 - 아니다. 특정 시군구 법정코드와 아파트 코드가 필요하다.
2. 특정 시군구 아파트 코드만 필요하다
 - 가능하려면 2.x 메가짜리 파일에서 특정 동을 검색해서 그부분만 잘라서 xml 로 만든 뒤에 매번 csv 임포트 해줘야 한다.

즉, sql 에 현재 가진 모든 법정코드와 주소를 테이블로 넣어두고 아파트 정보가 디테일하게 담긴 테이블에 정보를 넣어두는 것이 좋다.

**아파트 코드만 얻어오면 아파트 정보를 얻을 수 있다.**

- 그러나 읽어온 아파트 정보는 처리가 필요하다.
- 게다가 영문으로 된 정보가 한글로 무엇인지 번역될 필요가 있다.

```
http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255
```

In [2]:
  import ( "io/ioutil";"net/http";"strings";"fmt";"bytes" )

    res, err := http.Get("http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do?kapt_code=A10027255")
    defer res.Body.Close()
    body, err := ioutil.ReadAll(res.Body); buf := bytes.NewBuffer(body); json := buf.String(); fmt.Println(json[:300])
  	

{"resultMap_match":{"KAPT_CODE":"A10027255","TOWN_CODE":20306348},"resultMap_kapt":{"CODE_HEAT":"지역난방","SUBWAY_STATION":"-","SUBWAY_LINE":"7호선, 인천선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"분무식","KAPT_PE5AREA":0,"KAP
301
<nil>


# 구현 

## 모든 법정동 코드 정보를 테이블에 넣기

### 파일을 읽어서 존재하는 정보만 배열로 준비하기

1. 존재하는 법정동 정보 중
2. 코드 뒤 두자리가 00이면서
3. 이름이 00동인 것만 추려서
4. 코드 앞 8자리와 법정동명을 추출한다.

In [15]:
import ("encoding/csv"; "fmt"; "os"; "github.com/kniren/gota/dataframe";
       "bytes";"regexp";"strings")

// 접속 생성 및  닫기 지연 걸기
 csvFile, err := os.Open("./법정동코드_전체자료.txt")
 defer csvFile.Close()

// 일단 리더생성 후 쉼표 옵션 기호를 탭으로 설정
 reader := csv.NewReader(csvFile)
 reader.Comma = '\t' // Use tab-delimited instead of comma <---- here!
 reader.FieldsPerRecord = -1

// 모두 읽어제껴버림
 arrayData, err := reader.ReadAll()

// 헤더를 제외한 데이터만 배열에 담기
bjdArrData := [][]string{}
ind := 0

// 패턴 매칭 숫자 8개, 0 두개
r, _ := regexp.Compile("[0-9]{8}00")

for index, row := range arrayData{
    
    법정동코드 := row[0][:8]
    법정동명 := row[1]
    빈배열 := []string{법정동코드,법정동명 }
    존재여부 := row[2]
    숫자패턴 := r.MatchString(row[0])
    주소묶음 := strings.Fields(row[1])
    
                         
    if (index == 0){
        // bjdArrData = append(filteredArrData, row[:2])
        // index가 0 이면 다시 for 룹 시작으로 가서 index 1 부터 처리
        // 즉 헤더 빼고 데이터처리만 하라는 말
        continue
    }else if( 존재여부 == "존재" && 숫자패턴 && len(주소묶음) == 3 ){
        bjdArrData = append(bjdArrData, 빈배열)
    }
}

fmt.Println(bjdArrData[:50])

[[11110101 서울특별시 종로구 청운동] [11110102 서울특별시 종로구 신교동] [11110103 서울특별시 종로구 궁정동] [11110104 서울특별시 종로구 효자동] [11110105 서울특별시 종로구 창성동] [11110106 서울특별시 종로구 통의동] [11110107 서울특별시 종로구 적선동] [11110108 서울특별시 종로구 통인동] [11110109 서울특별시 종로구 누상동] [11110110 서울특별시 종로구 누하동] [11110111 서울특별시 종로구 옥인동] [11110112 서울특별시 종로구 체부동] [11110113 서울특별시 종로구 필운동] [11110114 서울특별시 종로구 내자동] [11110115 서울특별시 종로구 사직동] [11110116 서울특별시 종로구 도렴동] [11110117 서울특별시 종로구 당주동] [11110118 서울특별시 종로구 내수동] [11110119 서울특별시 종로구 세종로] [11110120 서울특별시 종로구 신문로1가] [11110121 서울특별시 종로구 신문로2가] [11110122 서울특별시 종로구 청진동] [11110123 서울특별시 종로구 서린동] [11110124 서울특별시 종로구 수송동] [11110125 서울특별시 종로구 중학동] [11110126 서울특별시 종로구 종로1가] [11110127 서울특별시 종로구 공평동] [11110128 서울특별시 종로구 관훈동] [11110129 서울특별시 종로구 견지동] [11110130 서울특별시 종로구 와룡동] [11110131 서울특별시 종로구 권농동] [11110132 서울특별시 종로구 운니동] [11110133 서울특별시 종로구 익선동] [11110134 서울특별시 종로구 경운동] [11110135 서울특별시 종로구 관철동] [11110136 서울특별시 종로구 인사동] [11110137 서울특별시 종로구 낙원동] [11110138 서울특별시 종로구 종로2가] [11110139 서울특별시 종로구 팔판동] [11110140 서울특별시 종로

### 테이블 생성 후 배열 자료를 추가하기 

생성

In [16]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

db, err := sql.Open("postgres", dbinfo)

if err != nil {
    log.Println(err)
}
defer db.Close()

droptable := `DROP TABLE IF EXISTS 법정동, 아파트정보;`

createtable:= `CREATE TABLE 법정동 (
                법정동코드 bigint NOT NULL DEFAULT 0,
                법정동명 varchar(30)  NOT NULL DEFAULT ''
              )`

// 쿼리 날릴 준비를 하고 실행한다.
stmt, err1 := db.Prepare(createtable)
// 쿼리 날릴 준비를 하고 실행한다.
stmt2, err1 := db.Prepare(droptable)



_, err = stmt2.Exec()
if err != nil {
    fmt.Println(err.Error())
}
defer stmt2.Close()

_, err = stmt.Exec()
if err != nil {
    fmt.Println(err.Error())
}
defer stmt.Close()

벌크 임포트(대량 가져오기) https://godoc.org/github.com/lib/pq#hdr-Bulk_imports

prepared statement http://go-database-sql.org/prepared.html


In [17]:
// 접속
db, err := sql.Open("postgres", dbinfo)
defer db.Close()


txn, err := db.Begin()
if err != nil {
	fmt.Println(err)
}
// func CopyIn(table string, columns ...string) string
stmt, err := txn.Prepare(pg.CopyIn("법정동", "법정동코드", "법정동명"))
if err != nil {
	fmt.Println(err)
}

for _, bjd := range bjdArrData {
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
        _, err = stmt.Exec(bjd[0], bjd[1])
	if err != nil {
		fmt.Println(err)
	}
}

_, err = stmt.Exec()
if err != nil {
	fmt.Println(err)
}

err = stmt.Close()
if err != nil {
	fmt.Println(err)
}

err = txn.Commit()
if err != nil {
	fmt.Println(err)
}

## 특정 키워드를 입력하면 해당하는 모든 법정동 코드 가져오는 함수 만들기

### 함수 설계 (mock)

In [33]:
func getBjd(areaName string) []int {
    
    법정동코드결과 := []int{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    
    법정동코드결과 = append(법정동코드결과, 1001, 1002, 1003)
    
    return 법정동코드결과
}

fmt.Println(getBjd("지역키워드"))

[1001 1002 1003]
17
<nil>


### 쿼리 설계

In [36]:
// db 접속 포인터를 얻고
db, err := sql.Open("postgres", dbinfo)

if err != nil { log.Println(err)}
defer db.Close()

// Query the database.
rows, err := db.Query(`
    SELECT 법정동코드
    FROM 법정동
    WHERE 법정동명 LIKE $1
    `, "%서울%")

if err != nil { log.Println(err)}
defer rows.Close()
    
fmt.Println("질의 날리기")

법정동코드결과 := []string{}

for rows.Next() {
    
    코드하나 := ""

if err := rows.Scan(&코드하나); err != nil {
log.Println(err)}
 
   법정동코드결과 =  append(법정동코드결과,코드하나)
// fmt.Printf("%.2d, \n", 법정동코드)
}
    
fmt.Println("출력하기", 법정동코드결과[:4])
    

질의 날리기
출력하기 [1100000000 1111000000 1111010100 1111010200]
59
<nil>


### 지역 키워드에 따라 법정동 코드를 가져오는 함수 완성

In [18]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

func getBjd(areaName string) [][]string {
    
    법정동코드결과 := [][]string{}
    
    // areaName Like 쿼리 결과 를 법정동 코드결과 변수에 append 로 넣는다.
    db, err := sql.Open("postgres", dbinfo)
    if err != nil { log.Println(err)}
    defer db.Close()
    
    // Query the database.
    rows, err := db.Query(`SELECT 법정동코드,법정동명 FROM 법정동 WHERE 법정동명 LIKE $1`,
                      "%"+ areaName+"%")
    index := 0
    for rows.Next() {    
        코드하나 := ""
        이름하나 := ""
        if err := rows.Scan(&코드하나,&이름하나); err != nil {log.Println(err)}
        한줄 := []string{코드하나,이름하나}
        법정동코드결과 =  append(법정동코드결과, 한줄)
        index += 1
    }
    
    return 법정동코드결과
}

bjdCode := getBjd("신길동")

fmt.Println(bjdCode)

[[11560132 서울특별시 영등포구 신길동]]
52
<nil>


## 법정동을 주면 아파트 코드를 가져오는 함수 작성

### 함수 설계 - Mock

In [5]:
func getAptCode(bjdCode int) []string {
    
    아파트코드결과 := []string{}
    
    // get 요청으로 반환받은 결과 값을 아파트 코드결과 변수에 append 로 넣는다.
    
    아파트코드결과 = append(아파트코드결과, "A1001", "A1002", "A1003")
    
    return 아파트코드결과
}

fmt.Println(getAptCode(263802006002))

[A1001 A1002 A1003]
20
<nil>


### get 요청과 처리 설계

계속 timeout 오류가 나서 내가 뭘 잘못했나. Timeout 시간까지 늘렸는데... 집 네트워크가 이상해서 그랬다. 현재 lte 네트워크로 작업하니 슉슉 잘 됨.

#### 요청 주소 만들고 GET 호출하기

직접 Get 을 썼다가. 아래처럼 request 요청으로 바꿈.
http://golang.site/go/article/102-HTTP-GET-호출

고 기본 get 은 실패시 재시도 기능이 없다.
https://github.com/sethgrid/pester

In [19]:
import (
    "fmt"
    "log"
    "net/http"
    "os"
 "net"
  "net/url"
    "strconv"
    "time"
    "io/ioutil"
    "github.com/sethgrid/pester"
)

func getAptlistRsp(targetUrl string, bjdCode string) []uint8{
    
    req, err := http.NewRequest("GET", targetUrl, nil)
    if err != nil {
        log.Print(err)
    }

    q := req.URL.Query()   
    q.Add("bjd_code",bjdCode) 
    req.URL.RawQuery = q.Encode()
    
    // url := req.URL.String()
    // fmt.Println(req.URL.String())
    
    
    // Client객체에서 Request 실행
//     client := &http.Client{Timeout: 40 * time.Second,}
//     resp, err := client.Do(req)
    
    // pester 사용
    resp, err := pester.Do(req)
    if err != nil {
        log.Println("error GETing example.com", err)
    }
    defer resp.Body.Close()
    log.Printf("example.com %s", resp.Status)
 
    // 결과 출력
    bytes, _ := ioutil.ReadAll(resp.Body)
    // str := string(bytes) //바이트를 문자열로
     // fmt.Println(str[:100])
    
    
    return  bytes
}
aptListurl := "http://www.k-apt.go.kr/kaptinfo/getKaptList.do"
respListBody := getAptlistRsp(aptListurl, bjdCode[0][0])

2018/03/22 21:24:38 example.com 200 OK


#### 주소 요청 결과로 구조체 만들기

구조는 다음과 같다.

In [ ]:
{
  "resultList": [
    {
      "KAPT_USEDATE": "199701",
      "BJD_CODE": "1135010500",
      "KAPT_NAME": "노원현대아파트",
      "OPEN_TERM": null, 
      "KAPT_CODE": "A13981604",
      "OCCU_FIRST_DATE": "199702",
      "X": 205515.315,
      "Y": 461795.5,
      "ENERGY_B_COUNT": 0,
      "BJD_NAME": "서울특별시 노원구 상계동"
    },
      ...
}

구조체는 다음과 같음..  

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/

네스티드로 구조체 짤 수도 있다.
https://stackoverflow.com/questions/32125816/parsing-xml-in-golang-unmarshaling

In [20]:
import(
    "encoding/json"
    "bytes"
"reflect")


type AptBasicResultList struct {
     결과목록 []AptBasicResult  `json:"resultList"`
}

type AptBasicResult struct{
    아파트사용일 string `json:"KAPT_USEDATE"`
    법정동코드 string  `json:"BJD_CODE"`
    아파트이름 string `json:"KAPT_NAME"`
    OPEN_TERM string `json:"OPEN_TERM"` // null 값을 타입으로 받을 수 있나?
    아파트코드 string `json:"KAPT_CODE"`
    완공첫날 int `json:"OCCU_FIRST_DATE"`
    x좌표 float32 `json:"X"`
    y좌표 float32 `json:"Y"`
    에너지_B_횟수 int `json:"ENERGY_B_COUNT"`
    법정동명 string  `json:"BJD_NAME"`
}


func unMarshalAptJson(responseBody []byte) AptBasicResultList {
    
    var data AptBasicResultList
    json.Unmarshal(responseBody, &data)

    fmt.Println(data.결과목록)
     jsonData, _ := json.Marshal(data)
     fmt.Println(string(jsonData))

 return data
}

xmlData := unMarshalAptJson(respListBody)


[{201704 1156013200 래미안에스티움  A10027073 0 191822.47 443975.94 0 서울특별시 영등포구 신길동} {201512 1156013200 래미안프레비뉴  A10027755 0 191811.52 444772.78 0 서울특별시 영등포구 신길동} {201307 1156013200 레전드힐스  A15005002 0 191828.02 444826.03 0 서울특별시 영등포구 신길동} {200508 1156013200 보라매경남아너스빌  A15086006 0 191981.3 443634.2 3 서울특별시 영등포구 신길동} {200511 1156013200 보라매두산위브  A15086001 0 192363.39 443896.1 0 서울특별시 영등포구 신길동} {200007 1156013200 신기목련  A15005102 0 192418.5 445133.9 0 서울특별시 영등포구 신길동} {198609 1156013200 신길1차우성아파트  A15005303 0 191125.22 444794.47 0 서울특별시 영등포구 신길동} {198509 1156013200 신길건영  A15005302 0 191201.55 444836.38 0 서울특별시 영등포구 신길동} {199906 1156013200 신길경남  A15083703 0 192731.19 445933.28 0 서울특별시 영등포구 신길동} {197412 1156013200 신길남서울  A15085805 0 192181.34 444453.8 0 서울특별시 영등포구 신길동} {200806 1156013200 신길뉴타운한화꿈에그린  A15005501 0 191935.7 444527.5 0 서울특별시 영등포구 신길동} {198712 1156013200 신길삼두  A15083701 0 193197.98 445793.12 0 서울특별시 영등포구 신길동} {198404 1156013200 신길삼성  A15005603 0 192095 443796.66 0 서울특별시 영등포구 신길동} {200106

#### XML 로부터 아파트 코드 추출하기

In [21]:
import(
    "encoding/json"
    "encoding/xml"
    "bytes"
"reflect"
"fmt")


func getAptCode(aptXmlData AptBasicResultList) []string {
        
    아파트코드결과 := []string{}
    
    // get 요청으로 반환받은 결과 값을 아파트 코드결과 변수에 append 로 넣는다.
    
    items := xmlData.결과목록
    
    for _,v := range items{
        // fmt.Println(k,v)
        아파트코드결과 = append(아파트코드결과, v.아파트코드 )
    }
    // fmt.Println(아파트코드결과[:10])
    
    return 아파트코드결과
}
aptCodes := getAptCode(xmlData)


## 아파트 기본 정보 담을 테이블 생성하기

### 아파트 기본 정보 xml 정보 확인


response body 만 xml 로 바꿔서 xml 파싱한다.
https://tutorialedge.net/golang/parsing-xml-with-golang/

https://www.thepolyglotdeveloper.com/2017/03/parse-xml-data-in-a-golang-application/


받게되는 정보에 관한 설명
https://www.data.go.kr/subMain.jsp#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTIkQF5wdWJsaWNEYXRhRGV0YWlsUGs9dWRkaTphMTJjNmU3Ni04OTgyLTRlM2UtYWZlMi05NDQxMmM3ZDBhNzkkQF5tYWluRmxhZz10cnVl

In [22]:
import(
    "encoding/json"
    "encoding/xml"
    "io/ioutil"
    "bytes"
    "net/http"
    "reflect"
    "log"
    "fmt"
    "github.com/sethgrid/pester"
)

func getAptDetailtRsp(targetUrl string, kaptCode string) []byte{
    
    req, err := http.NewRequest("GET", targetUrl, nil)
    if err != nil {
        log.Print(err)
        //os.Exit(1)
    }

    q := req.URL.Query()  
    q.Add("kapt_code",  kaptCode)
    req.URL.RawQuery = q.Encode()
    // url := req.URL.String()
//     fmt.Println(req.URL.String())
//     return  url   
    
        // Client객체에서 Request 실행
//     client := &http.Client{Timeout: 40 * time.Second,}
//     resp, err := client.Do(req)
    
    // pester 사용
    resp, err := pester.Do(req)
    if err != nil {
        log.Println("error GETing example.com", err)
    }
    defer resp.Body.Close()
    log.Printf("example.com %s", resp.Status)
 
    // 결과 출력
    bytes, _ := ioutil.ReadAll(resp.Body)
    str := string(bytes) //바이트를 문자열로
    fmt.Println(str)
    
    
    return  bytes
    
}

aptinfourl := "http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do"
repDetailBody := getAptDetailtRsp(aptinfourl, aptCodes[0])

{"resultMap_match":{"KAPT_CODE":"A10027073","TOWN_CODE":20349595},"resultMap_kapt":{"CODE_HEAT":"개별난방","SUBWAY_STATION":"신풍역","SUBWAY_LINE":"7호선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"도포식,분무식","KAPT_PE5AREA":0,"KAPTD_ECNTB":0,"KAPTD_ECNTC":0,"KAPT_PHAREA":0,"KAPTD_ECNTD":0,"KAPTD_SCNT":"16","KAPTD_ECNTE":0,"CODE_ECON":"종합계약","KAPT_PPAREA":0,"KAPTD_CCCNT":"206","KAPTD_ECNTM":0,"KAPT_TEL":"028499571","WELFARE_FACILITY":"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤니티공간, 자전거보관소","KAPTD_ECNTP":52,"KAPT_FAX":"02-849-9574","KAPT_ACC_COMPANY":"이지스엔터프라이즈","KAPT_MGR_CNT":"14","KAPT_BCOMPANY":"삼성물산주식회사","CONVENIENT_FACILITY":"-","CODE_DISINF":"위탁관리","CODE_EMGR":"상주선임","CODE_MGR":"위탁관리","KAPTD_PCNTU":"2280","CODE_EMAINT":"위탁관리","KAPT_MCAREA":44964.64,"CODE_ELEV":"위탁관리","KAPT_MEAREA":0,"CODE_NET":"유","KAPT_MHAREA":0,"KAPT_USEDATE":"2017-04-12","KAPTD_WTIMEBUS":"5분이내","KAPT_CODE":"A10027073","CODE_APT":"아파트","CODE_WSUPPLY":"부스타방식","KAP

2018/03/22 21:24:40 example.com 200 OK


In [23]:
type AptDetailResponse struct {
    매치정보 AptDetailItemMatch `json:"resultMap_match"`
    세부정보 AptDetailItem `json:"resultMap_kapt"`
    세대정보 []AptDetailSedae `json:"resultMap_kapt_areacnt"`
    주소정보 []AptDetailAddress`json:"resultMap_kapt_addrList"`
}

type AptDetailItemMatch struct {
    아파트코드 string `json:"KAPT_CODE"`
    마을코드 int `json:"TOWN_CODE"`
}

type AptDetailSedae struct {
    세대수 int `json:"KAPTDA_CNT"`
    전용면적구분 int `json:"AREA_GBN"`// 60㎡ 이하: 60㎡ ~ 85㎡ 이하: 85㎡ ~ 135㎡ 이하: 135㎡ 초과:
    아파트코드 string `json:"KAPT_CODE"`
}

type AptDetailAddress struct {
    법정동주소 string `json:"ADDR"`
    아파트코드 string `json:"KAPT_CODE"`
    주소구분 string `json:"ADDR_GBN"`
}

type AptDetailItem struct {
    아파트이름 	string `json:"KAPT_NAME"`
    분양형태	string `json:"CODE_SALE"`
    난방방식	string `json:"CODE_HEAT"`
    연면적	float64 `json:"KAPT_TAREA"`
    동수 string `json:"KAPT_DONG_CNT"`
    시공사 string `json:"KAPT_BCOMPANY"`
    시행사 string `json:"KAPT_ACOMPANY"`
    관리사무소연락처	string `json:"KAPT_TEL"`
    관리사무소팩스	string `json:"KAPT_FAX"`
    홈페이지주소	string `json:"KAPT_URL"`
    단지분류	string `json:"CODE_APT"`
    
    일반관리방식 string `json:"CODE_MGR"`
    일반관리인원 	string `json:"KAPT_MGR_CNT"`
    일반관리계약업체  	string `json:"KAPT_CCOMPANY"` 
    
    경비관리방식	string `json:"CODE_SEC"`
    경비관리인원	string `json:"KAPTD_SCNT"`
    경비관리계약업체 	string `json:"KAPTD_SEC_COM"`

    청소관리방식	string `json:"CODE_CLEAN"`
    청소관리인원	string `json:"KAPTD_CLCNT"`
    음식물처리방법 	string `json:"CODE_GARBAGE"`
    
    소독관리방식	string `json:"CODE_DISINF"`
    연간소독횟수 	string `json:"KAPTD_DCNT"`
    소독방법	string `json:"DISPOSAL_TYPE"`
    
    지상주차장대수 string `json:"KAPTD_PCNT"`
    지하주차장대수 string `json:"KAPTD_PCNTU"`
    
    cctv대수  string `json:"KAPTD_CCCNT"`
    부대복리시설  string `json:"WELFARE_FACILITY"`
    수전용량  string `json:"KAPTD_ECAPA"`
    전기안전관리자법정선임여부  	string `json:"CODE_EMGR"`
    급수방식  string `json:"CODE_WSUPPLY"`
    
        건물구조  string `json:"CODE_STR"`
    세대전기계약방식 string `json:"CODE_ECON"`
    화재수신반방식  string `json:"CODE_FALARM"`
    
    승강기관리형태  string `json:"CODE_ELEV"`
    // 승객용승강기  string `json:"KAPTD_ECNTP"`
    //화물용승강기  string `json:"KAPTD_PCNT"`
    //비상승강기 string `json:"KAPTD_DCNT"`

    주차관제홈네트워크  	string `json:"CODE_NET"`
    
    버스정류장 	string `json:"KAPTD_WTIMEBUS"`
    편의시설  	string `json:"CONVENIENT_FACILITY"`
    교육시설  	string `json:"EDUCATION_FACILITY"`
    
    복도유형	string `json:"CODE_HALL"`
    사용승인일	string `json:"KAPT_USEDATE"`
    주거전용면적 float64 `json:"KAPT_MAREA"`
    
    인근지하철역	string `json:"SUBWAY_STATION"`
    인근지하철호선	string `json:"SUBWAY_LINE"`
    지하철까지시간 	string `json:"KAPTD_WTIMESUB"`
    아파트코드	string `json:"KAPT_CODE"`
    
 }


func unMarshalAptDetail(responseBody []byte) AptDetailResponse {
    
    var data AptDetailResponse
    json.Unmarshal([]byte(responseBody), &data)
    
     // fmt.Println(data.세부정보.명칭)
    jsonData, _ := json.Marshal(data)
    fmt.Println(string(jsonData))

 return data
}

xmlData := unMarshalAptDetail(repDetailBody)

{"resultMap_match":{"KAPT_CODE":"A10027073","TOWN_CODE":20349595},"resultMap_kapt":{"KAPT_NAME":"래미안에스티움","CODE_SALE":"임대+분양","CODE_HEAT":"개별난방","KAPT_TAREA":272491.57,"KAPT_DONG_CNT":"19","KAPT_BCOMPANY":"삼성물산주식회사","KAPT_ACOMPANY":"신길7재정비촉진구역주택재개발사업조함","KAPT_TEL":"028499571","KAPT_FAX":"02-849-9574","KAPT_URL":"estium.co.kr","CODE_APT":"아파트","CODE_MGR":"위탁관리","KAPT_MGR_CNT":"14","KAPT_CCOMPANY":"(주)타워피엠씨","CODE_SEC":"위탁관리","KAPTD_SCNT":"16","KAPTD_SEC_COM":"(주)씨큐원","CODE_CLEAN":"위탁관리","KAPTD_CLCNT":"23","CODE_GARBAGE":"음식물쓰레기종량제","CODE_DISINF":"위탁관리","KAPTD_DCNT":"12","DISPOSAL_TYPE":"도포식,분무식","KAPTD_PCNT":"0","KAPTD_PCNTU":"2280","KAPTD_CCCNT":"206","WELFARE_FACILITY":"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤니티공간, 자전거보관소","KAPTD_ECAPA":"7550","CODE_EMGR":"상주선임","CODE_WSUPPLY":"부스타방식","CODE_STR":"철골콘크리트구조","CODE_ECON":"종합계약","CODE_FALARM":"R형","CODE_ELEV":"위탁관리","CODE_NET":"유","KAPTD_WTIMEBUS":"5분이내","CONVENIENT_FACILITY":"-","EDUCATION_FACILITY":"-","CODE_HALL":"혼합식","KAPT_USED

### 쿼리 작성 + 테이블 생성

In [24]:
createtable:= 
`CREATE TABLE 아파트정보 (
              
    아파트코드 varchar(10)  NOT NULL DEFAULT '',
    아파트이름 	varchar(51)  NOT NULL DEFAULT '',
    마을코드 bigint NOT NULL DEFAULT 0,
    법정동주소 varchar(200) NOT NULL DEFAULT 0,
    도로명주소 varchar(200) NOT NULL DEFAULT 0,
    
    단지분류	varchar(51)  NOT NULL DEFAULT '',
    
    동수 varchar(100)  NOT NULL DEFAULT '',    
    세대수 bigint NOT NULL DEFAULT 0,
    
    전용면적60이하아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적60이하세대수  bigint NOT NULL DEFAULT 0,
    전용면적60이상85이하아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적60이상85이하세대수  bigint NOT NULL DEFAULT 0,
    전용면적85이상135이하아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적85이상135이하세대수  bigint NOT NULL DEFAULT 0,
    전용면적135초과아파트코드 varchar(52)  NOT NULL DEFAULT '',
    전용면적135초과세대수  bigint NOT NULL DEFAULT 0,
    
    분양형태	varchar(52)  NOT NULL DEFAULT '',
    난방방식	varchar(52)  NOT NULL DEFAULT '',
    복도유형	varchar(52)  NOT NULL DEFAULT '',

    연면적	double precision  NOT NULL DEFAULT 0,

    시공사 varchar(100)  NOT NULL DEFAULT '',
    시행사 varchar(100)  NOT NULL DEFAULT '',
    
    관리사무소연락처	varchar(53)  NOT NULL DEFAULT '',
    관리사무소팩스	varchar(53)  NOT NULL DEFAULT '',
    홈페이지주소	varchar(100)  NOT NULL DEFAULT '',
    
    일반관리방식 varchar(54)  NOT NULL DEFAULT '',
    일반관리인원 varchar(52)  NOT NULL DEFAULT '',
    일반관리계약업체  	varchar(100)  NOT NULL DEFAULT '',
    
    경비관리방식	varchar(54)  NOT NULL DEFAULT '',
    경비관리인원	varchar(52)  NOT NULL DEFAULT '',
    경비관리계약업체 	varchar(100)  NOT NULL DEFAULT '',

    청소관리방식	varchar(54)  NOT NULL DEFAULT '',
    청소관리인원	varchar(52)  NOT NULL DEFAULT '',
    음식물처리방법 	varchar(100)  NOT NULL DEFAULT '',
    
    소독관리방식	varchar(54)  NOT NULL DEFAULT '',
    연간소독횟수  varchar(54)  NOT NULL DEFAULT '',
    소독방법	varchar(54)  NOT NULL DEFAULT '',
    
    승강기관리형태  	varchar(55)  NOT NULL DEFAULT '',
    승객용승강기 varchar(54)  NOT NULL DEFAULT '',
    화물용승강기 varchar(54)  NOT NULL DEFAULT '',
    비상승강기 varchar(54)  NOT NULL DEFAULT '',
    주차관제홈네트워크  	varchar(58)  NOT NULL DEFAULT '',

    건물구조  	varchar(100)  NOT NULL DEFAULT '',
    세대전기계약방식  	varchar(55)  NOT NULL DEFAULT '',
    화재수신반방식  	varchar(55)  NOT NULL DEFAULT '',

    
    지상주차장대수 varchar(57)  NOT NULL DEFAULT '',
    지하주차장대수 varchar(57)  NOT NULL DEFAULT '',
    
    cctv대수  	varchar(57)  NOT NULL DEFAULT '',
    부대복리시설  	varchar(70)  NOT NULL DEFAULT '',
    수전용량  	varchar(57)  NOT NULL DEFAULT '',
    전기안전관리자법정선임여부  	varchar(57)  NOT NULL DEFAULT '',
    급수방식  	varchar(57)  NOT NULL DEFAULT '',
    

    
    버스정류장 	varchar(51)  NOT NULL DEFAULT '',
    편의시설  	varchar(100)  NOT NULL DEFAULT '',
    교육시설  	varchar(100)  NOT NULL DEFAULT '',
    

    사용승인일	varchar(51)  NOT NULL DEFAULT '',
    주거전용면적 double precision  NOT NULL DEFAULT 0,
    
    인근지하철역	varchar(50)  NOT NULL DEFAULT '',
    인근지하철호선	varchar(50)  NOT NULL DEFAULT '',
    지하철까지시간 	varchar(50)  NOT NULL DEFAULT ''


)`

In [25]:
import (
	"database/sql"
	"log"
	"os"
    "fmt"
    
    pg "github.com/lib/pq"
)

    _ = pg.Efatal

const (
    DB_USER     = "gopher"
    DB_PASSWORD = "1111"
    DB_NAME     = "gopher" // postgres create DB named created user's
    DB_HOST        = "db"
)
 
dbinfo := fmt.Sprintf("user=%s password=%s dbname=%s host=%s sslmode=disable",
    DB_USER, DB_PASSWORD, DB_NAME, DB_HOST)

db, err := sql.Open("postgres", dbinfo)

if err != nil {
    log.Print(err)
}
defer db.Close()

// droptable := `DROP TABLE IF EXISTS 법정동, 아파트정보;`

// stmt2, err1 := db.Prepare(droptable)
// _, err = stmt2.Exec()
// if err != nil {
//     fmt.Println(err.Error())
// }
// defer stmt2.Close()

// 쿼리 날릴 준비를 하고 실행한다.
stmt, err1 := db.Prepare(createtable)

_, err = stmt.Exec()
if err != nil {
    fmt.Print(err.Error())
}
defer stmt.Close()

## 특정 주소 키워드를 주면 지역 내 아파트정보를 반환하는 함수 만들기

In [26]:
import "strconv"

// 지역 키워드를 주면, 아파트 정보들을 반환한다.
func getAptBaseInfo(areaKeyword string) []AptDetailResponse {
    
    
    // prepare const
    aptListurl := "http://www.k-apt.go.kr/kaptinfo/getKaptList.do"
    //aptinfourl := "http://www.k-apt.go.kr/kaptinfo/getKaptInfo_detail.do"

    
    // bucket
    var aptInformations []AptDetailResponse
    
    
    fmt.Println("법정동 목록 시작","-----------")
    
    // 법정동 주소의 목록을 반환
    법정동목록 := getBjd(areaKeyword)
    
    fmt.Println(법정동목록)
    
    fmt.Println("법정동 목록 완료","-----------")
    
    // 법정동 하나씩 마다
    for _, bjdCode := range 법정동목록{

        bjdCode := bjdCode[0]

        fmt.Println(bjdCode,"아파트 목록 조회 시작","-----------")
        
        respListBody := getAptlistRsp(aptListurl, bjdCode)
        // _ = respBody
        xmlData := unMarshalAptJson(respListBody)
        // _ = xmlData
        
        fmt.Println("아파트 목록 조회 완료","-----------")
        
        fmt.Println("아파트 목록 마샬링 시작","-----------")
        
        // 여러 아파트 코드를 알아내서
        aptCodes := getAptCode(xmlData)

        fmt.Println("아파트 목록 마샬링 완료","-----------")
        
        // 아파트 코드 하나마다
        fmt.Println("아파트 정보 조회 시작","-----------")
        
        for index, kaptCode := range aptCodes{

            rspDetailBody := getAptDetailtRsp(aptinfourl, kaptCode)
            xmlData := unMarshalAptDetail(rspDetailBody)
            
            // 정보를 알아낸 다음 차곡차곡 끼워 넣는다.
            aptInformations = append(aptInformations, xmlData)
            fmt.Println(  index+1,"/" ,len(aptCodes) ,xmlData.세부정보.아파트이름, "처리완료")
        }
        
        fmt.Println(len(aptInformations),"개 아파트 처리 완료")
    }

    return aptInformations
}

aptDetailInfo := getAptBaseInfo("오류동")

// 결과가 없는데 출력하려고 하면 범위 경계 오류 남.
// panic: runtime error: slice bounds out of range
fmt.Println(aptDetailInfo)

법정동 목록 시작 -----------
[[11530108 서울특별시 구로구 오류동] [28245118 인천광역시 계양구 오류동] [28260119 인천광역시 서구 오류동] [30140113 대전광역시 중구 오류동]]
법정동 목록 완료 -----------
11530108 아파트 목록 조회 시작 -----------
[{200310 1153010800 경남아너스빌  A15210207 0 185770.56 443150.5 0 서울특별시 구로구 오류동} {199903 1153010800 동부(돌타운)아파트  A15210103 0 186564.98 443897.62 0 서울특별시 구로구 오류동} {201802 1153010800 서울 오류동 행복주택  A10026434 0 186434.3 443732.7 0 서울특별시 구로구 오류동} {198712 1153010800 서울가든빌라  A15289508 0 184989.83 443353.7 0 서울특별시 구로구 오류동} {199601 1153010800 오류금강  A15210206 0 186476.28 443434 0 서울특별시 구로구 오류동} {200309 1153010800 오류금강수목원  A15210211 0 185110.44 442893.4 0 서울특별시 구로구 오류동} {200609 1153010800 오류금호어울림  A15210102 0 185705.83 444039.4 0 서울특별시 구로구 오류동} {200207 1153010800 오류동 영풍마드레빌  A15210212 0 185674.33 443271.53 0 서울특별시 구로구 오류동} {199812 1153010800 오류동부골든  A15210107 0 185763.39 444119.2 0 서울특별시 구로구 오류동} {199709 1153010800 오류동부골든SH임대  A15278301 0 185763.39 444119.2 0 서울특별시 구로구 오류동} {199807 1153010800 오류동부덕고개  A15210105 0 186474.52 44388

2018/03/22 21:24:42 example.com 200 OK


{"resultMap_match":{"KAPT_CODE":"A10027073","TOWN_CODE":20349595},"resultMap_kapt":{"CODE_HEAT":"개별난방","SUBWAY_STATION":"신풍역","SUBWAY_LINE":"7호선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"도포식,분무식","KAPT_PE5AREA":0,"KAPTD_ECNTB":0,"KAPTD_ECNTC":0,"KAPT_PHAREA":0,"KAPTD_ECNTD":0,"KAPTD_SCNT":"16","KAPTD_ECNTE":0,"CODE_ECON":"종합계약","KAPT_PPAREA":0,"KAPTD_CCCNT":"206","KAPTD_ECNTM":0,"KAPT_TEL":"028499571","WELFARE_FACILITY":"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤니티공간, 자전거보관소","KAPTD_ECNTP":52,"KAPT_FAX":"02-849-9574","KAPT_ACC_COMPANY":"이지스엔터프라이즈","KAPT_MGR_CNT":"14","KAPT_BCOMPANY":"삼성물산주식회사","CONVENIENT_FACILITY":"-","CODE_DISINF":"위탁관리","CODE_EMGR":"상주선임","CODE_MGR":"위탁관리","KAPTD_PCNTU":"2280","CODE_EMAINT":"위탁관리","KAPT_MCAREA":44964.64,"CODE_ELEV":"위탁관리","KAPT_MEAREA":0,"CODE_NET":"유","KAPT_MHAREA":0,"KAPT_USEDATE":"2017-04-12","KAPTD_WTIMEBUS":"5분이내","KAPT_CODE":"A10027073","CODE_APT":"아파트","CODE_WSUPPLY":"부스타방식","KAP

2018/03/22 21:24:42 example.com 200 OK


{"resultMap_match":{"KAPT_CODE":"A10027755","TOWN_CODE":20331410},"resultMap_kapt":{"CODE_HEAT":"개별난방","SUBWAY_STATION":"-","SUBWAY_LINE":"7호선","KAPT_PE1AREA":0,"KAPT_PE2AREA":0,"KAPT_PE3AREA":0,"KAPT_PEAREA":0,"KAPT_PE4AREA":0,"DISPOSAL_TYPE":"-","KAPT_PE5AREA":0,"KAPTD_ECNTB":0,"KAPTD_ECNTC":0,"KAPT_PHAREA":0,"KAPTD_ECNTD":0,"KAPTD_SCNT":"11","KAPTD_ECNTE":0,"CODE_ECON":"종합계약","KAPT_PPAREA":0,"KAPTD_CCCNT":"100","KAPTD_ECNTM":0,"KAPT_TEL":"02-849-6655","WELFARE_FACILITY":"관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 커뮤니티공간, 자전거보관소","KAPTD_ECNTP":28,"KAPT_FAX":"02-849-6900","KAPT_ACC_COMPANY":"-","KAPT_MGR_CNT":"8","KAPT_BCOMPANY":"삼성물산주식회사","CONVENIENT_FACILITY":"-","CODE_DISINF":"위탁관리","CODE_EMGR":"상주선임","CODE_MGR":"위탁관리","KAPTD_PCNTU":"1211","CODE_EMAINT":"위탁관리","KAPT_MCAREA":23340.24,"CODE_ELEV":"위탁관리","KAPT_MEAREA":0,"CODE_NET":"유","KAPT_MHAREA":0,"KAPT_USEDATE":"2015-12-29","KAPTD_WTIMEBUS":"5~10분이내","KAPT_CODE":"A10027755","CODE_APT":"아파트","CODE_WSUPPLY":"부스타방식","KAPT_MPAREA":688620.8

...

[{{A10027073 20349595} {래미안에스티움 임대+분양 개별난방 272491.57 19 삼성물산주식회사 신길7재정비촉진구역주택재개발사업조함 028499571 02-849-9574 estium.co.kr 아파트 위탁관리 14 (주)타워피엠씨 위탁관리 16 (주)씨큐원 위탁관리 23 음식물쓰레기종량제 위탁관리 12 도포식,분무식 0 2280 206 관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤니티공간, 자전거보관소 7550 상주선임 부스타방식 철골콘크리트구조 종합계약 R형 위탁관리 유 5분이내 - - 혼합식 2017-04-12 133313.49 신풍역 7호선 5분이내 A10027073} [{682 1 A10027073} {833 2 A10027073} {207 3 A10027073} {0 4 }] [{서울특별시 영등포구 신길동 4950 래미안에스티움 A10027073 B} {서울특별시 영등포구 신풍로 77 A10027073 R}]} {{A10027755 20331410} {래미안프레비뉴 임대+분양 개별난방 140645.79 12 삼성물산주식회사 조합 02-849-6655 02-849-6900 - 아파트 위탁관리 8 (주)아주관리 위탁관리 11 (주)아주관리 위탁관리 10 거점장비수거방식 위탁관리 0 - 0 1211 100 관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 커뮤니티공간, 자전거보관소 4000 상주선임 부스타방식 철골철근콘크리트구조 종합계약 R형 위탁관리 유 5~10분이내 - 초등학교() 중학교() 고등학교() 혼합식 2015-12-29 68800.87 - 7호선 5~10분이내 A10027755} [{396 1 A10027755} {535 2 A10027755} {17 3 A10027755} {0 4 }] [{서울특별시 영등포구 신길동 329-94 래미안영등포프레비뉴 A10027755 B} {서울특별시 영등포구 가마산로 442 A10027755 R}]} {{A15005002 20181

수조식 철근콘크리트구조 종합계약 P형 위탁관리 무 5~10분이내 관공서(동사무소) 병원(성애병원) 대형상가(LG하이프라자-대방점) 공원(여의도샛강생태공원) 중학교(윤중중학교) 고등학교(한국제과고등기술학교) 복도식 1987-12-12 7333.2 대방 1호선, 5호선 5분이내 A15083701} [{160 1 A15083701} {0 2 } {0 3 } {0 4 }] [{서울특별시 영등포구 신길동 2-3 삼두아파트 A15083701 B} {서울특별시 영등포구 여의대방로61길 19 A15083701 R}]} {{A15005603 1052} {신길삼성 분양 개별난방 30549.74 2 삼성종합건설(주) 삼성종합건설(주) 02-846-2273 02-846-2275 - 아파트 위탁관리 5 율산개발(주) 위탁관리 4 희망코리아(주) 위탁관리 3 음식물쓰레기종량제 위탁관리 12 도포식,분무식,독이식 218 0 41 관리사무소, 노인정, 어린이놀이터, 자전거보관소 700 상주선임 고가수조식 철근콘크리트구조 단일계약 P형 위탁관리 무 5~10분이내 관공서(동사무소) 병원(원치과) 대형상가(LG하이프라자) 공원(보라매공원) 기타(돈보스코청소년센터) 초등학교(대길초등학교) 중학교(대방중학교) 고등학교(영신고등학교) 복도식 1984-04-14 21766.32 신풍 7호선 5~10분이내 A15005603} [{120 1 A15005603} {264 2 A15005603} {0 3 } {0 4 }] [{서울특별시 영등포구 신길동 4759 삼성아파트 A15005603 B} {서울특별시 영등포구 여의대방로 45 A15005603 R}]} {{A15005402 1053} {신길삼성래미안 분양 개별난방 111955.89 12 삼성물산 신길2-3지구주택재개발조합 02-832-7937 02-832-7938 - 아파트 위탁관리 8 서일개발 위탁관리 13 (주)엘림테크 위탁관리 8 음식물쓰레기종량제 위탁관리 12 도포식,분무식,독이식 122 853 126 관리사무소, 노인정, 보육시설, 주민공동시설

4 }] [{서울특별시 영등포구 신길동 4935 신기목련아파트 A15005102 B} {서울특별시 영등포구 신길로42가길 34 A15005102 R}]} {{A15005303 1057} {신길1차우성아파트 분양 개별난방 61767.2 6 (주)우성건설 (주)우성건설 02-833-6584 02-6673-6583 apt913.aptgate.co.kr 아파트 위탁관리 5 (주)아주관리 위탁관리 9 조은세이프 위탁관리 6 음식물쓰레기종량제 위탁관리 12 도포식,분무식,독이식 470 0 48 관리사무소, 노인정, 어린이놀이터 1760 상주선임 고가수조식 철근콘크리트구조 단일계약 P형 위탁관리 무 5분이내 관공서(동사무소) 병원(한산부인과) 대형상가(LG하이프라자) 공원(근린공원) 초등학교(대영초등학교) 중학교(대영중교) 고등학교(대영고교) 혼합식 1986-09-03 46238.48 신도림역,문래역 1호선, 2호선 5~10분이내 A15005303} [{182 1 A15005303} {506 2 A15005303} {0 3 } {0 4 }] [{서울특별시 영등포구 신길동 365 신길우성아파트 A15005303 B} {서울특별시 영등포구 도신로 68-1 A15005303 R}]} {{A15005302 1047} {신길건영 분양 개별난방 44462.75 4 LIG건영 (주)건영 844-2535 844-2535 신길건영.apti.co.kr 아파트 자치관리 4 - 자치관리 6 - 위탁관리 5 음식물쓰레기종량제 위탁관리 12 분무식,독이식 313 0 107 관리사무소, 노인정, 어린이놀이터, 자전거보관소 995 위탁선임 부스타방식 철근콘크리트구조 종합계약 P형 위탁관리 유 5~10분이내 관공서(신길3동) 병원(한산부인과) 대형상가(LG하이프라자) 공원(근린공원) 초등학교(대영초등학교) 중학교(대영중교) 고등학교(대영고교) 복도식 1985-09-04 31387.44 신풍역 1호선, 2호선, 7호선 5~10분이내 A15005302} [{0 1 } {160 2 A15005302} {22

5.08 대방, 신풍 1호선, 7호선 5~10분이내 A15085404} [{172 1 A15085404} {144 2 A15085404} {104 3 A15085404} {0 4 }] [{서울특별시 영등포구 신길동 769-1 한성아파트 A15085404 B} {서울특별시 영등포구 가마산로 575 A15085404 R}]} {{A10027073 20349595} {래미안에스티움 임대+분양 개별난방 272491.57 19 삼성물산주식회사 신길7재정비촉진구역주택재개발사업조함 028499571 02-849-9574 estium.co.kr 아파트 위탁관리 14 (주)타워피엠씨 위탁관리 16 (주)씨큐원 위탁관리 23 음식물쓰레기종량제 위탁관리 12 도포식,분무식 0 2280 206 관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 휴게시설, 커뮤니티공간, 자전거보관소 7550 상주선임 부스타방식 철골콘크리트구조 종합계약 R형 위탁관리 유 5분이내 - - 혼합식 2017-04-12 133313.49 신풍역 7호선 5분이내 A10027073} [{682 1 A10027073} {833 2 A10027073} {207 3 A10027073} {0 4 }] [{서울특별시 영등포구 신길동 4950 래미안에스티움 A10027073 B} {서울특별시 영등포구 신풍로 77 A10027073 R}]} {{A10027755 20331410} {래미안프레비뉴 임대+분양 개별난방 140645.79 12 삼성물산주식회사 조합 02-849-6655 02-849-6900 - 아파트 위탁관리 8 (주)아주관리 위탁관리 11 (주)아주관리 위탁관리 10 거점장비수거방식 위탁관리 0 - 0 1211 100 관리사무소, 노인정, 보육시설, 문고, 주민공동시설, 어린이놀이터, 커뮤니티공간, 자전거보관소 4000 상주선임 부스타방식 철골철근콘크리트구조 종합계약 R형 위탁관리 유 5~10분이내 - 초등학교() 중학교() 고등학교() 혼합식 2015-12-29 68800.87 - 7호선 5~

포구 대방천로 180 A15086007 R}]} {{A15086004 1059} {신길우성3차아파트 분양 개별난방 48820 5 우성건설 우성건설 02-848-3078 02-845-0811 ws3apt@hanmail.net 아파트 자치관리 5 - 자치관리 9 - 위탁관리(직영+위탁) 3 음식물쓰레기종량제 위탁관리(직영+위탁) 4 분무식 192 0 60 관리사무소, 노인정, 어린이놀이터, 자전거보관소 950 위탁선임 고가수조식 철근콘크리트구조 종합계약 P형 위탁관리(직영+위탁) 무 5~10분이내 관공서(동사무소) 병원(이화의원) 대형상가(하이마트-신풍점) 공원(보라매공원) 기타(돈보스코청소년센터) 초등학교(대길초등학교) 고등학교(영신고등학교) 혼합식 1989-04-11 33366.24 신풍 7호선 5~10분이내 A15086004} [{139 1 A15086004} {338 2 A15086004} {0 3 } {0 4 }] [{서울특별시 영등포구 신길동 4656 신길우성3차아파트 A15086004 B} {서울특별시 영등포구 대방천로14길 32 A15086004 R}]} {{A15005001 1060} {신길우성4차 분양 개별난방 53207.29 4 우성건설 우성건설 02)849-1156 02)849-1157 우성4차.aptgate.co.kr 아파트 위탁관리 5 (주)디엠지관리 기타 10 디엠지관리 기타 5 음식물쓰레기종량제 기타 12 분무식 203 89 88 관리사무소, 노인정, 어린이놀이터, 자전거보관소 1400 상주선임 고가수조식 철근콘크리트구조 단일계약 P형 기타 무 5분이내 관공서(동사무소) 병원(서울대윤병원) 대형상가(삼성디지털프라자) 공원(영등포공원) 중학교(영원중교) 고등학교(영등포여고교, 장훈고등학교) 계단식 1991-05-04 38900 신길 1호선, 5호선 5분이내 A15005001} [{0 1 } {476 2 A15005001} {0 3 } {0 4 }] [{서울특별시 영등포구 신길동 102-8 신길우성아파트 A15005001 B} {서울특별시

## 테이블에 자료를 넣고 원하는 것만 꺼내서 csv 파일로 출력하기

자료를 모두 담을 구조체 생성 // 이럴 바에야 json flatten 시키고 string 을 한 구조체에 담을 걸 ㅠㅠ

In [27]:
열이름 := []string{
"아파트코드",
"마을코드",
"법정동주소",
"도로명주소",
"세대수",
"전용면적60이하세대수",
"전용면적60이상85이하세대수",
"전용면적85이상135이하세대수",
"전용면적135초과세대수",
"전용면적60이하아파트코드",
"전용면적60이상85이하아파트코드",
"전용면적85이상135이하아파트코드",
"전용면적135초과아파트코드",
"아파트이름",
"분양형태",
"난방방식",
"연면적",
"동수",
"시공사",
"시행사",
"관리사무소연락처",
"관리사무소팩스",
"홈페이지주소",
"단지분류",
"일반관리방식",
"일반관리인원",
"일반관리계약업체",
"경비관리방식",
"경비관리인원",
"경비관리계약업체",
"청소관리방식",
"청소관리인원",
"음식물처리방법",
"소독관리방식",
"연간소독횟수",
"소독방법",
"건물구조",
"세대전기계약방식",
"화재수신반방식",
"승강기관리형태",
"지상주차장대수",
"지하주차장대수",
"cctv대수",
"부대복리시설",
"수전용량",
"전기안전관리자법정선임여부",
"급수방식",
"승객용승강기",
"화물용승강기",
"비상승강기",
"주차관제홈네트워크",
"버스정류장",
"편의시설",
"교육시설",
"복도유형",
"사용승인일",
"주거전용면적",
"인근지하철역",
"인근지하철호선",
"지하철까지시간",
}

```sql
매치 := apt.매치정보
세부 := apt.세부정보
세대1 := apt.세대정보[0]
세대2 := apt.세대정보[1]
세대3 := apt.세대정보[2]
세대4 := apt.세대정보[3]
주소 := apt.주소정보[0]

stmt.Exec(
    세부.아파트코드,
    매치.마을코드,
    주소.법정동주소,
    주소.주소구분,
    세대.세대수,
    세대1.세대수,
    세대2.세대수,
    세대3.세대수,
    세대4.세대수,
    세대1.아파트코드,
    세대2.아파트코드,
    세대3.아파트코드,
    세대4.아파트코드,
    세부.아파트이름,
    세부.분양형태,
    세부.난방방식,
    세부.연면적,
    세부.동수,
    세부.시공사,
    세부.시행사,
    세부.관리사무소연락처,
    세부.관리사무소팩스,
    세부.홈페이지주소,
    세부.단지분류,
    세부.일반관리방식,
    세부.일반관리인원,
    세부.일반관리계약업체,
    세부.경비관리방식,
    세부.경비관리인원,
    세부.경비관리계약업체,
    세부.청소관리방식,
    세부.청소관리인원,
    세부.음식물처리방법,
    세부.소독관리방식,
    세부.연간소독횟수,
    세부.소독방법,
    세부.건물구조,
    세부.세대전기계약방식,
    세부.화재수신반방식,
    세부.승강기관리형태,
    세부.지상주차장대수,
    세부.지하주차장대수,
    세부.cctv대수,
    세부.부대복리시설,
    세부.수전용량,
    세부.전기안전관리자법정선임여부,
    세부.급수방식,
    세부.승객용승강기,
    세부.화물용승강기,
    세부.비상승강기,
    세부.주차관제홈네트워크,
    세부.버스정류장,
    세부.편의시설,
    세부.교육시설,
    세부.복도유형,
    세부.사용승인일,
    세부.주거전용면적,
    세부.인근지하철역,
    세부.인근지하철호선,
    세부.지하철까지시간,
)
```

In [28]:

func writeAptInfo(aptDetailInfo []AptDetailResponse) {
// 접속
db, err := sql.Open("postgres", dbinfo)
defer db.Close()


txn, err := db.Begin()
if err != nil {
	fmt.Println(err)
}
// func CopyIn(table string, columns ...string) string
stmt, err := txn.Prepare(pg.CopyIn("아파트정보", 열이름...))
if err != nil {
	fmt.Println(err)
}

for _, apt := range aptDetailInfo {
    
    매치 := apt.매치정보
    세부 := apt.세부정보
    
    // 세대정보와 주소가 null 로 응답되는 경우 처리
    if len(apt.세대정보) ==4 &&  len(apt.주소정보) ==2 {
     
    세대1 := apt.세대정보[0]
    세대2 := apt.세대정보[1]
    세대3 := apt.세대정보[2]
    세대4 := apt.세대정보[3]
    주소1 := apt.주소정보[0]
    주소2 := apt.주소정보[1]

    
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
    _, err = stmt.Exec(
        세부.아파트코드,
        매치.마을코드,
        주소1.법정동주소,
        주소2.법정동주소,
        세대1.세대수+세대2.세대수+세대3.세대수+세대4.세대수,
        세대1.세대수,
        세대2.세대수,
        세대3.세대수,
        세대4.세대수,
        세대1.아파트코드,
        세대2.아파트코드,
        세대3.아파트코드,
        세대4.아파트코드,
        세부.아파트이름,
        세부.분양형태,
        세부.난방방식,
        세부.연면적,
        세부.동수,
        세부.시공사,
        세부.시행사,
        세부.관리사무소연락처,
        세부.관리사무소팩스,
        세부.홈페이지주소,
        세부.단지분류,
        세부.일반관리방식,
        세부.일반관리인원,
        세부.일반관리계약업체,
        세부.경비관리방식,
        세부.경비관리인원,
        세부.경비관리계약업체,
        세부.청소관리방식,
        세부.청소관리인원,
        세부.음식물처리방법,
        세부.소독관리방식,
        세부.연간소독횟수,
        세부.소독방법,
        세부.건물구조,
        세부.세대전기계약방식,
        세부.화재수신반방식,
        세부.승강기관리형태,
        세부.지상주차장대수,
        세부.지하주차장대수,
        세부.cctv대수,
        세부.부대복리시설,
        세부.수전용량,
        세부.전기안전관리자법정선임여부,
        세부.급수방식,
        "",
        "",
        "",
        세부.주차관제홈네트워크,
        세부.버스정류장,
        세부.편의시설,
        세부.교육시설,
        세부.복도유형,
        세부.사용승인일,
        세부.주거전용면적,
        세부.인근지하철역,
        세부.인근지하철호선,
        세부.지하철까지시간,
    )
    }else if(len(apt.세대정보) ==4 &&  len(apt.주소정보) ==1){
        
        세대1 := apt.세대정보[0]
    세대2 := apt.세대정보[1]
    세대3 := apt.세대정보[2]
    세대4 := apt.세대정보[3]
    주소1 := apt.주소정보[0]

    
//     _, err = stmt.Exec(int64(bjd[0]), string(bjd[1]))
    _, err = stmt.Exec(
        세부.아파트코드,
        매치.마을코드,
        주소1.법정동주소,
        "",
        세대1.세대수+세대2.세대수+세대3.세대수+세대4.세대수,
        세대1.세대수,
        세대2.세대수,
        세대3.세대수,
        세대4.세대수,
        세대1.아파트코드,
        세대2.아파트코드,
        세대3.아파트코드,
        세대4.아파트코드,
        세부.아파트이름,
        세부.분양형태,
        세부.난방방식,
        세부.연면적,
        세부.동수,
        세부.시공사,
        세부.시행사,
        세부.관리사무소연락처,
        세부.관리사무소팩스,
        세부.홈페이지주소,
        세부.단지분류,
        세부.일반관리방식,
        세부.일반관리인원,
        세부.일반관리계약업체,
        세부.경비관리방식,
        세부.경비관리인원,
        세부.경비관리계약업체,
        세부.청소관리방식,
        세부.청소관리인원,
        세부.음식물처리방법,
        세부.소독관리방식,
        세부.연간소독횟수,
        세부.소독방법,
        세부.건물구조,
        세부.세대전기계약방식,
        세부.화재수신반방식,
        세부.승강기관리형태,
        세부.지상주차장대수,
        세부.지하주차장대수,
        세부.cctv대수,
        세부.부대복리시설,
        세부.수전용량,
        세부.전기안전관리자법정선임여부,
        세부.급수방식,
        "",
        "",
        "",
        세부.주차관제홈네트워크,
        세부.버스정류장,
        세부.편의시설,
        세부.교육시설,
        세부.복도유형,
        세부.사용승인일,
        세부.주거전용면적,
        세부.인근지하철역,
        세부.인근지하철호선,
        세부.지하철까지시간,
    )
}else {
            _, err = stmt.Exec(
        세부.아파트코드,
        매치.마을코드,
        "",
        "",
        0,
        0,
        0,
        0,
        0,
        "",
        "",
        "",
        "",
        세부.아파트이름,
        세부.분양형태,
        세부.난방방식,
        세부.연면적,
        세부.동수,
        세부.시공사,
        세부.시행사,
        세부.관리사무소연락처,
        세부.관리사무소팩스,
        세부.홈페이지주소,
        세부.단지분류,
        세부.일반관리방식,
        세부.일반관리인원,
        세부.일반관리계약업체,
        세부.경비관리방식,
        세부.경비관리인원,
        세부.경비관리계약업체,
        세부.청소관리방식,
        세부.청소관리인원,
        세부.음식물처리방법,
        세부.소독관리방식,
        세부.연간소독횟수,
        세부.소독방법,
        세부.건물구조,
        세부.세대전기계약방식,
        세부.화재수신반방식,
        세부.승강기관리형태,
        세부.지상주차장대수,
        세부.지하주차장대수,
        세부.cctv대수,
        세부.부대복리시설,
        세부.수전용량,
        세부.전기안전관리자법정선임여부,
        세부.급수방식,
        "",
        "",
        "",
        세부.주차관제홈네트워크,
        세부.버스정류장,
        세부.편의시설,
        세부.교육시설,
        세부.복도유형,
        세부.사용승인일,
        세부.주거전용면적,
        세부.인근지하철역,
        세부.인근지하철호선,
        세부.지하철까지시간,
    )
    }
        
	if err != nil {
		log.Println(err)
	}
}

_, err = stmt.Exec()
if err != nil {
	fmt.Println(err)
}

err = stmt.Close()
if err != nil {
	fmt.Println(err)
}

err = txn.Commit()
if err != nil {
	fmt.Println(err)
}
    
}

writeAptInfo(aptDetailInfo)

In [25]:
import (
    "reflect"
    "fmt"
    "log"
    "os"
    "github.com/kniren/gota/dataframe"
    "github.com/kniren/gota/series"   
)
reflect.TypeOf(x)

df := dataframe.LoadStructs(x[0].세대정보)
fmt.Println(df)

# /tmp/go-build925817822/libgithub.com-yunabe-lgo-sess7b2274696d65223a313532313730303736353436363735393230317d-exec25.so
github.com/yunabe/lgo/sess7b2274696d65223a313532313730303736353436363735393230317d/exec25.lgo_init: missing section for relocation target type.sz/KTjha
github.com/yunabe/lgo/sess7b2274696d65223a313532313730303736353436363735393230317d/exec25.lgo_init: reloc 26 to non-elf symbol type.sz/KTjha (outer=type.sz/KTjha) 0
github.com/yunabe/lgo/sess7b2274696d65223a313532313730303736353436363735393230317d/exec25.lgo_init: undefined: "type.sz/KTjha"
Failed to build a shared library of github.com/yunabe/lgo/sess7b2274696d65223a313532313730303736353436363735393230317d/exec25: exit status 2
